In [ ]:
import os
import pytz


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

MY_TIMEZONE = pytz.timezone("Asia/Ho_Chi_Minh")
DATASET_DIR = r"./dataset"
CHECKPOINT_DIR = r"./checkpoint"
WORKING_DIR = os.getcwd()

In [ ]:
import yaml
from yaml.loader import SafeLoader
with open("NIDS.yaml") as f:
    config = yaml.load(f, Loader=SafeLoader)

config['dataset']['path'] = [DATASET_DIR + "/netflowv1/NF-NF-ToN-IoT.csv",]

In [ ]:
match config['server']['model']:
    case 'CNNmodel':
        TASK = 'b'
        N_CLASS = 1
    case 'DeepFedmodel':
        TASK = 's'
        N_CLASS = 2
    case 'FedTHmodel':
        TASK = 'b'
        N_CLASS = 1
    case 'NFHuntermodel':
        TASK = 'b'
        N_CLASS = 1
    case _:
        raise ValueError("Somethings wrong.")

In [ ]:
from mdlp2.data.pipe.fed import fed_pipe
from mdlp2.data.pipe.nfv1 import nfv1_pipe

In [ ]:
TEST_SIZE = config['dataset']['test_size']
NUM_CLIENTS = config['server']['n_client']

fed_pipeline = fed_pipe(source_type="CSV", split_radio=TEST_SIZE, n_client=NUM_CLIENTS, keep_radio_feature='Label', random_state=123, verbose=False)
test_data, client_data = fed_pipeline(config['dataset']['path'])

nfv1_pipeline = nfv1_pipe(source_type="DataFrame", task=TASK, verbose=False)
np_test_data = nfv1_pipeline(test_data)
np_client_data = [nfv1_pipeline(data) for data in client_data]

In [ ]:
import numpy as np
def sequence_split(input_np, sequence_length:int=16, keep_final:bool=False):
    seq_num = len(input_np)//sequence_length
    seqs = [input_np[i*sequence_length:(i+1)*sequence_length] for i in range(seq_num)]
    return np.stack(seqs, axis=0)

In [ ]:
np_test_data = [sequence_split(x) for x in np_test_data]

for i in range(len(np_client_data)):
    np_client_data[i] = [sequence_split(x) for x in np_client_data[i]]

In [ ]:
import tensorflow as tf
import mdlp2.model as model_list

In [ ]:
optim = tf.keras.optimizers.legacy.Adam(config['client']['compile']['lr'])
metric = ['Accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
if np_test_data[1].shape[-1] == 1:
    loss = tf.keras.losses.BinaryCrossentropy()
elif np_test_data[1].shape[-1] >= 2:
    loss = tf.keras.losses.CategoricalCrossentropy()
else:
    raise ValueError(f"Data have an invalid number of classes ({client_data[1].shape[-1]}).")

In [ ]:
model = getattr(model_list, config['server']['model'])(input_shape=np_test_data[0].shape[1:], n_classes=N_CLASS)
model.compile(optimizer=optim, loss=loss, metrics=metric)

In [ ]:
from mdlp2.fed import fed_avg_simulator

In [ ]:
global_W, simulation_history = fed_avg_simulator("NIDS.yaml", np_test_data, np_client_data, model)

In [ ]:
import pickle

if not os.path.exists(os.path.join(CHECKPOINT_DIR, f"./train/{config['server']['model']}")):
    os.makedirs(os.path.join(CHECKPOINT_DIR, f"./train"),exist_ok=True)
    

with open(os.path.join(CHECKPOINT_DIR, f"./train/{config['server']['model']}.pkl"), 'wb') as f:
    pickle.dump([global_W, simulation_history], f)


In [ ]:
print("Loss Accuracy Precision Recall F1-Score")
for k, v in simulation_history[1][1].items():
    print("{:.4f}".format(v[0].numpy()), end=" ")
    print("{:.4f} {:.4f} {:.4f} {:.4f}".format(*v[1:]))